In [11]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRanker
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Подготовка тренировочного датасета
def prepare_training_data(df):
    rows = []
    groups = []
    for _, row in df.iterrows():
        if row['winner'] == 0:
            rows.append({'r': row['r1'], 'd': row['d1'], 'score': 1})
            rows.append({'r': row['r2'], 'd': row['d2'], 'score': 0})
            groups.append(2)
        elif row['winner'] == 1:
            rows.append({'r': row['r1'], 'd': row['d1'], 'score': 0})
            rows.append({'r': row['r2'], 'd': row['d2'], 'score': 1})
            groups.append(2)
    return pd.DataFrame(rows), groups

# Загрузка и подготовка данных
train_df = pd.read_csv('restaurants_train.txt', sep='\t', names=['winner', 'r1', 'r2', 'd1', 'd2'])
train_data, group = prepare_training_data(train_df)
X_train = train_data[['r', 'd']]
y_train = train_data['score']

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Обучение модели ранжирования
model = LGBMRanker(objective='lambdarank', metric='ndcg')
model.fit(X_train_scaled, y_train, group=group)

# Функция предсказания для тестового набора
def predict_scores(test_df):
    X_test = test_df[['r', 'd']]
    X_test_scaled = scaler.transform(X_test)
    return model.predict(X_test_scaled)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000080 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 1752, number of used features: 2


In [9]:
print("Веса (коэффициенты):", model.coef_)  # Выведет веса для каждого признака
print("Смещение (intercept):", model.intercept_) 

Веса (коэффициенты): [0.01703038 0.        ]
Смещение (intercept): 0.4253210158050861


In [8]:
def predict(X_test):
    r_w = 0.01703038
    d_w = 0.0
    bias = 0.4253210158050861
    bias = np.array([bias for i in range(len(X_test))])
    y_pred = r_w*X_test[:,0] + d_w*X_test[:, 1] + bias
    return y_pred

with open('restaurants.in', 'r') as file:
    
    n = int(file.readline().strip()) 
    cafes = []
    
    for _ in range(n):
        r, d = map(float, file.readline().strip().split())
        cafes.append([r, d])

y_pred = predict(np.array(cafes))
for i in range(len(y_pred)):
    print(y_pred[i])

FileNotFoundError: [Errno 2] No such file or directory: 'restaurants.in'

[1 4]
[3 6]


array([ 9, 24])